In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import xgboost
from sklearn.ensemble import RandomForestClassifier

In [ ]:
file_path = '../input/heart-failure-clinical-data'
heart_data = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')
heart_data.head()

In [ ]:
heart_data.describe()

***Checking for null values:***

In [ ]:
heart_data.isna().sum()

We are in luck as we do not have any missing values so we do not have to fill in the dummy values

# **1. EDA of the data**

In [ ]:
#We plot the distribution of the age of people in the dataset:
plt.figure(figsize=(10,6))
sns.histplot(data= heart_data, x = "age", kde=True)


In [ ]:
# We plot the distribution of the gender of the people coming in:
sns.kdeplot(data = heart_data,x= heart_data.age, hue='sex', shade=True)

In [ ]:
#To find out the density of the population between male and female genders divided into whether they smoke or not
plt.figure(figsize=(5,5))
sns.kdeplot(data = heart_data, x = heart_data.sex,hue='smoking', shade=True)

***We can see that there is no difference between the gender of the people diagnosed with heart disease.***

In [ ]:
# #Checking out the number of smokers between genders(Assuming 1 for male and 0 for female gender)
male = heart_data[heart_data["sex"]==1]
female = heart_data[heart_data["sex"]==0]

male_smoker = male[heart_data["smoking"]==1]
female_smoker = female[heart_data["smoking"]==1]
smokers = [len(male_smoker),len(female_smoker)]
sns.barplot(y=smokers, x = ['Male smokers','Female Smokers'])

As we can see that the male population has 

In [ ]:
male_survi = male[heart_data["DEATH_EVENT"]==0]
male_not = male[heart_data["DEATH_EVENT"]==1]
female_survi = female[heart_data["DEATH_EVENT"]==0]
female_not = female[heart_data["DEATH_EVENT"]==1]

labels = ['Male - Survived','Male - Not Survived', "Female -  Survived", "Female - Not Survived"]
values = [len(male_survi),len(male_not),
         len(female_survi),len(female_not)]
plt.figure(figsize=(7,7))
def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%  ({v:d})'.format(p=pct,v=val)
    return my_autopct

plt.pie(values, labels= labels, autopct=make_autopct(values),  labeldistance=1)
# plt.legend()
plt.show() 

We would now drill down even more to differentiate between the smokers and non-smokers between genders and find out their survival.

In [ ]:
# Checking among the survivers:
male_survi_smoker = male_survi[male_survi['smoking']==1]
male_survi_non_smoker = male_survi[male_survi.smoking==0]

female_survi_smoker = female_survi[female_survi.smoking==1]
female_survi_non_smoker = female_survi[female_survi.smoking==0]

labels = ['Male - Smokers','Male - Non-Smokers', 'Female -  Smokers', 'Female - Non smokers']
values = [len(male_survi_smoker),len(male_survi_non_smoker),len(female_survi_smoker), len(female_survi_non_smoker)]


# plt.subplot(3,4,1)
plt.pie(values, labels= labels, autopct=make_autopct(values),  labeldistance=1 ,radius=2)
plt.title("Distribution of survivers")


We can see that among the survivers there isnot much difference between the smokers and the no-smokers for the male population but almost all the female population survivers are non-smokers. 

In [ ]:
male_not_survi_smoker = male_not[male_not['smoking']==1]
male_not_survi_non_smoker = male_not[male_not['smoking']==0]
female_not_survi_smoker = female_not[female_not['smoking']==1]
female_not_survi_non_smoker = female_not[female_not['smoking']==0]
labels_not = ['Male - Smokers','Male - Non-Smokers', 'Female -  Smokers', 'Female - Non smokers']
values_not_survived = [len(male_not_survi_smoker),len(male_not_survi_non_smoker),len(female_not_survi_smoker), len(female_not_survi_non_smoker)]
plt.pie(values_not_survived,labels= labels_not, autopct=make_autopct(values_not_survived),  labeldistance=1, radius=2)
plt.title("Distribution among the people who did not survive")


In [ ]:
sns.swarmplot(data=heart_data, x = 'DEATH_EVENT', y = 'creatinine_phosphokinase',)

There is no particular relationship between the levels of creatinine_phosphokinase and Death Event.

# **2. We will now start with our Data Modelling:**
# A. Random Forrest Regressor

In [ ]:
heart_data.head()

In [ ]:
Features = ['time','ejection_fraction','serum_creatinine','platelets']
y = heart_data['DEATH_EVENT']
heart_data.drop(['DEATH_EVENT'], axis = 1)

We would not drop any data now and look at how the model performs. Since we do not have any explanation about time we dropped that column.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,mean_absolute_error,mean_squared_error


In [ ]:
X_copy = heart_data[Features]
y_copy = y.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_copy, y_copy, train_size=0.7, random_state=0, shuffle = True)
model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)
preds = model.predict(X_test)
ac = accuracy_score(y_test, preds)
mae = mean_absolute_error(preds,y_test)
print("Using Random Forrest Classifier:")
print(f'Model accuracy is: {ac*100}%')
print(f'Mean absolute error is: {mae}')


#  B. XG BOOST:

In [ ]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=100, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)
predictions = my_model.predict(X_test)
xg_ac = my_model.score(X_test, y_test)
print('For XG BOOST model')
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))
print(f'The model score for XG BOOST model is: {xg_ac}')


> **AS you can see that XG BOOST did not perform better than Random Forrest Classifier. Therefore we go with the Random Forrest model.**

# ***Please let me know in the comments what do you think of my code. Thanks.***